This notebook explores both model index and opc ua scripts and contain examples of all the functions to make request to model index api and opc ua api servers.  

### Import Libraries

In [ ]:
# Import the required packeages
import pandas as pd
from typing import List, Dict
import sys
import os
import asyncio
import datetime

### Import Scripts

In [ ]:
# Setting the path is only needed if you are running this locally from source.
# Consider using pip to install instead.
module_path = os.path.abspath(os.path.join("../src/pyprediktormapclient/"))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# Import model index functions
from pyprediktormapclient.model_index import ModelIndex

# Import OPC UA functions
from pyprediktormapclient.opc_ua import OPC_UA

# Import Analytics Helper
from pyprediktormapclient.analytics_helper import AnalyticsHelper

# Import "Dataframer" Tools
from pyprediktormapclient.shared import *

In [ ]:
# Connection to the servers
model_index_url = "http://10.100.59.152:13371/v1/"
opcua_rest_url = "https://apis-opcua-api.pview.dev/"
opcua_server_url = "opc.tcp://81.166.54.193:4853"

# Model index API
model = ModelIndex(url=model_index_url)

# OPC UA API
tsdata = OPC_UA(rest_url=opcua_rest_url, opcua_url= opcua_server_url)

In [ ]:
# Input parameters for value data
# Parameters for aggregate historical data
start_time = (datetime.datetime.now() - datetime.timedelta(30)).strftime('%Y-%m-%dT%H:%M:%SZ') # 30 days ago
end_time = (datetime.datetime.now() - datetime.timedelta(29)).strftime('%Y-%m-%dT%H:%M:%SZ') # 29 days ago, so one day of data
pro_interval = 3600000 # 10 minutes processing time
agg_name = "Average"

### Download data from modelindex api

In [ ]:
# Listed sites on the model index api server
namespaces = model.get_namespace_array()
namespaces

In [ ]:
# Types of Objects
object_types_json = model.get_object_types()
object_types = AnalyticsHelper(obj_types_json)
object_types.dataframe

In [ ]:
# Unique types of Objects
object_types_unique = object_types.dataframe[['Id', 'Name']].drop_duplicates()
object_types_unique

In [ ]:
# To get typeId by type name of an object
object_type_id = model.get_object_type_id_from_name("SiteType")
object_type_id

In [ ]:
# To get the objects of a type
sites_json = model.get_objects_of_type("SiteType")

# Send the returned JSON into a normalizer to get Id, Type, Name, Props and Vars as columns
sites = AnalyticsHelper(sites_json)
sites.list_of_names()

In [ ]:
# Analytics helper
sites.variables_as_dataframe()

In [ ]:
sites.list_of_ids()

In [ ]:
# String sets for all parks
string_sets_json = model.get_object_descendants("StringSetType", sites.list_of_ids(), "PV_Assets")
string_sets = AnalyticsHelper(string_sets_json).dataframe
string_sets

In [ ]:
# Selecting the first site
first_site_id = sites.list_of_ids()[0]

In [ ]:
# Get all stringsets for one park
string_sets_for_first_park_as_json = model.get_object_descendants("StringSetType", [first_site_id], "PV_Assets")
string_sets_for_first_park = AnalyticsHelper(string_sets_for_first_park_as_json)
string_sets_for_first_park.dataframe

In [ ]:
# Ancestors of an object type, get all trackers that are ancestor of the parks string sets

trackers_as_json = model.get_object_ancestors("TrackerType", string_sets_for_first_park.list_of_ids(), "PV_Serves")
trackers = AnalyticsHelper(trackers_as_json)
trackers.variables_as_dataframe()

### Download data from the opc ua api

In [ ]:
# Live value data of trackers 
live_value = tsdata.get_live_values_data(['AngleMeasured', 'AngleSetpoint'], trackers.dataframe)
live_value

In [ ]:
# 1 day aggregated historical trackers data
await tsdata.get_agg_hist_value_data(start_time, end_time, pro_interval, agg_name, trackers.dataframe, ['AngleMeasured', 'AngleSetpoint'])

In [ ]:
# Reading downloaded tracker data 
tracker_df = pd.read_parquet('Data/data_chunk_0.parquet')
tracker_df